In [1]:
from collections import OrderedDict
import pickle as pkl
import os

import seaborn as sns
import pandas as pd
import numpy as np

from auxiliary import get_propensity_scores_matching_demonstration_4
from auxiliary import get_sample_matching_demonstration_2
from auxiliary import get_sample_matching_demonstration_3
from auxiliary import get_sample_matching_demonstration_4
from auxiliary import get_sparsity_pattern_by_treatment
from auxiliary import get_sparsity_pattern_overall
from auxiliary import get_propensity_score_3
from auxiliary import plot_propensity_score
from auxiliary import get_common_support
from auxiliary import get_lalonde_data
from auxiliary import get_inv_odds
from auxiliary import plot_weights
from auxiliary import get_odds

np.random.seed(123)

# Lecture 5: Matching estimators of causal effects

## Introduction

There exists only one back-door path $D \leftarrow S \leftrightarrow X \rightarrow Y$ and both $S$ nor $X$ are observable. Thus, we have a choice to condition on either one of them.

<img src="material/fig-conditioning-balance-adjust.png" height=300 width=300 />

* $X$, regression estimator, adjustment-for-other-causes conditioning strategy
* $S$, matching estimator, balancing conditioning strategy

**Agenda**

* matching as conditioning via stratification
* matching as weighting
* matching as data analysis algorithm

**Fundamental concepts**

* stratification of data
* weighting to achieve balance
* propensity scores

**Views on matching**

* method to form quasi-experimental contrasts by sampling comparable treatment and control cases
* nonparametric method of adjustment for treatment assignment patterns

**Simulation data**

The simulated data is inspired by real-world applications and thus rather complex. Nevertheless, the will serve as examples for several of the upcoming lectures. That is why we will invest some time initially to set up one of them in details.

## Matching as conditioning via stratification

Individuals within groups determined by $S$ are entirely indistinguishable from each other in all ways except 

* observed treatment status

* differences in potential outcomes that are independent of treatment status

More formally, we are able to assert the following **conditional independence assumptions**.

\begin{align*}
E[Y^1 \mid D = 1, S] = E[Y^1 \mid D = 0, S] \\
E[Y^0 \mid D = 1, S] = E[Y^0 \mid D = 0, S]
\end{align*}

implied by ...

* treatment assignment is ignorable
* selection on observables

**ATC**

\begin{align*}
E[\delta \mid D = 0, S] & = E[Y^1 - Y^0 \mid D = 0, S] \\
                        & = E[Y^1 \mid D = 0, S] - E[Y^0 \mid D = 0, S] \\
                        & = E[Y^1 \mid D = 1, S] - E[Y^0 \mid D = 0, S] \\
                        & = E[Y \mid D = 1, S] - E[Y \mid D = 0, S] \\
\end{align*}

**ATT**
\begin{align*}
E[\delta \mid D = 1, S] & = E[Y^1 - Y^0 \mid D = 1, S] \\
                        & = E[Y^1 \mid D = 1, S] - E[Y^0 \mid D = 1, S] \\
                        & = E[Y^1 \mid D = 1, S] - E[Y^0 \mid D = 0, S] \\
                        & = E[Y \mid D = 1, S] - E[Y \mid D = 0, S] \\
\end{align*}

<img src="material/fig-matching-demonstration-1.png" height=500 width=500 />

**Features**

* The gains from treatment participation differ in each stratum and those that have the most to gain are more likely to participate. So unconditional independence between $D$ and $(Y^1, Y^2)$ does not hold.

Let's study these idealized conditions for a simulated dataset.

Let us see our simulation in action.

We are in the comfortable position to not only compute the naive estimate but also the true average treatment effect.

What to do?

Note that the observed outcomes within each stratum correspond to the average potential outcome 
within the stratum. We can compute the average treatment effect by looking at the difference within each strata.

The ATT and ATC can be computed analogously just by applying the appropriate weights to the strata-specific effect of treatment.

More generally.

\begin{align*}
\{E_N [y_i \mid d_i = 1, s = s_i] - E_N [y_i \mid d_i = 0, s = s_i]\} \\
\xrightarrow{p} E[Y^1 - Y^0\mid S = s] = E[\delta \mid S = s].
\end{align*}
Weighted sums of these stratified estimates can then be taken such as for the unconditional ATE:
\begin{align*}
& \sum_s \{E_N[y_i \mid d_i = 1, s_i = s] - E_N[y_i \mid d_i = 0, s_i = s]\} \\
& * {\Pr}_N[s_i = s] \xrightarrow{p} E[\delta]
\end{align*}


This examples shows all of the basic principles in matching estimators that we will discuss in 
greater detail in this lecture. 

* Treatment and control subjects are matched together in the sense that they are grouped together 
into strata.

* An average difference between the outcomes of the treatment and control subjects is estimated, 
based on a weighting of the strata by common distribution.

### Overlap conditions

In [60]:
df = get_sample_matching_demonstration_2(num_agents=1000)
df[["Y", "D", "S"]].head()

,Y,D,S
0,3.220843,0,1
1,15.505549,1,3
2,0.814226,0,1
3,1.453293,0,1
4,2.939864,0,1


In [61]:
df.groupby(["S", "D"])["Y"].mean()

S  D
1  0     1.986109
2  0     5.854302
   1     8.184356
3  0    10.245960
   1    13.978689
Name: Y, dtype: float64

What can we do?

## Matching as weighting

As indicated by the stylized example, there are often many strata where we do not have treated and control individuals available at the same time.

$\rightarrow$ combine information from different strata with the same propensity score $p$

**Definition** The estimated propensity score is the estimated probability of taking the treatment as a function of variables that predict treatment assignment, i.e. $\Pr[D = 1 \mid S]$.

$\rightarrow$ stratifying on the propensity score itself ameliorates the sparseness problem because the propensity score can be treated as a single stratifying variable.

In [ ]:
a_grid = np.linspace(0.01, 1.00, 100)
b_grid = np.linspace(0.01, 1.00, 100)

df, counts = get_sample_matching_demonstration_3(a_grid, b_grid)
df.head()

**underlying causal graphs**

<img src="material/fig-matching-demonstration-3.png" height=500 width=500 />

We will now look at different ways to construct estimates for the usual causal parameters. So, we first compute their true counterparts.

In [ ]:
true_effects = list()
true_effects += [(df["y_1"] - df["y_0"])[(df["d"] == 1)].mean()]
true_effects += [(df["y_1"] - df["y_0"])[(df["d"] == 0)].mean()]
true_effects += [(df["y_1"] - df["y_0"]).mean()]

In [ ]:
print(
    "The true estimate of the average causal effect is {:5.3f}".format(
        true_effects[-1]
    )
)

stat = df["y"][df["d"] == 1].mean() - df["y"][df["d"] == 0].mean()
print(
    "The naive estimate of the average causal effect is {:5.3f}".format(stat)
)

How about the issue of sparsity on the data?

In [ ]:
get_sparsity_pattern_overall(counts)

In [ ]:
get_sparsity_pattern_by_treatment(counts)

How does the propensity score $P(D = 1\mid S)$ as a function of the observables $(a, b)$ look like?

In [ ]:
plot_propensity_score(a_grid, b_grid)

We still must be worried about common support.

In [ ]:
get_common_support(df)


\begin{align*}
\hat{\delta}_{\text{ATT, weight}} \equiv \left( \frac{1}{n^1}\sum_{i:d_i = 1} y_i\right) 
- \left(\frac{\sum_{i:d_i=0}\hat{r}_i y_i}{\sum_{i:d_i = 0} \hat{r}_i}\right)
\end{align*}

\begin{align*}
\hat{\delta}_{\text{ATC, weight}} \equiv 
\left(
\frac{\sum_{i: d_i = 1}\frac{y_i}{\hat{r}_i}}{\sum_{i: d_i = 1}\frac{1}{\hat{r}_i}}
\right)
- \left(\frac{1}{n^0} \sum_{i: d_i = 0} y_i\right) 
\end{align*}

\begin{align*}
\hat{\delta}_{\text{ATE, weight}} \equiv \left(\frac{1}{n}\sum_{i}d_i\right) \hat{\delta}_{\text{ATT, weight}} +  \left(1 - \frac{1}{n}\sum_{i}d_i\right) \hat{\delta}_{\text{ATC, weight}}
\end{align*}



**Weights** 

\begin{align*}
r_i = \frac{p_i}{1 - p_i}
\end{align*}

In [ ]:
plot_weights()

In [ ]:
# This example is important as it introduces students to the
# actual setup for the estimation of the propensity score
# and its potential misspecification.
def get_att_weight(df, p):
    """ Get weighted ATT.
    
    Calculates the weighted ATT basd on a provided
    dataset and the propensity score.
    
    Args:
        df: A dataframe with the observed data.
        p: A numpy array with the weights.
        
    Returns:
        A float which corresponds to the ATT.
    """
    weights = get_odds(p)

    is_control = df["d"] == 0
    is_treated = df["d"] == 1

    value, weights = df["y"][is_control], weights[is_control]
    att = df["y"][is_treated].mean() - np.average(value, weights=weights)

    return att


def get_atc_weight(df, p):
    """ Get weighted ATC.
    
    Calculates the weighted ATC basd on a provided
    dataset and the propensity score.
    
    Args:
        df: A dataframe with the observed data.
        p: A numpy array with the weights.
        
    Returns:
        A float which corresponds to the ATC.
    """
    weights = get_inv_odds(p)

    is_control = df["d"] == 0
    is_treated = df["d"] == 1

    value, weights = df["y"][is_treated], weights[is_treated]
    atc = np.average(value, weights=weights) - df["y"][is_control].mean()

    return atc


def get_ate_weight(df, p):
    """ Get weighted ATE.
    
    Calculates the weighted ATE basd on a provided
    dataset and the propensity score.
    
    Args:
        df: A dataframe with the observed data.
        p: A numpy array with the weights.
        
    Returns:
        A float which corresponds to the ATE.
    """
    share_treated = df["d"].value_counts(normalize=True)[1]

    atc = get_atc_weight(df, p)
    att = get_att_weight(df, p)

    return share_treated * att + (1.0 - share_treated) * atc


rslt = dict()
for model in ["true", "correct", "misspecified"]:
    p = get_propensity_score_3(df, model)

    rslt[model] = list()
    rslt[model] += [get_att_weight(df, p)]
    rslt[model] += [get_atc_weight(df, p)]
    rslt[model] += [get_ate_weight(df, p)]

    print("")
    print(model.capitalize())
    print(
        "estimated: ATT {:5.3f} ATC {:5.3f} ATE {:5.3f}".format(*rslt[model])
    )
    print(
        "true:      ATT {:5.3f} ATC {:5.3f} ATE {:5.3f}".format(*true_effects)
    )

## Matching as data analysis algorithm

\begin{align*}
\hat{\delta}_{\text{ATT, match}} = \frac{1}{n^1} \sum_i \left[
(y_i \mid d_i = 1) - \sum_j \omega_{i, j} (y_j \mid d_j =0 )
\right]
\end{align*}

\begin{align*}
\hat{\delta}_{\text{ATC, match}} = \frac{1}{n^0} \sum_j \left[
 \sum_i \omega_{j, i} (y_i\mid d_i = 1) - (y_j \mid d_j = 0)
\right]
\end{align*}

### Basic variants

* exact matching

    * construct counterfactual based on individuals with identical $S$


* nearest-neighbor and caliper

    * construct counterfactual based on individuals closest on a unidimensional measure, caliper ensures reasonable maximum distance

* interval matching

    * construct counterfactual by sorting individuals into segments based on unidimensional metric


* kernel matching

    * constructs counterfactual based on all individuals but weights them cased on the distance

### Benchmarking tutorial

In [ ]:
df = get_sample_matching_demonstration_4()
df.head()

In [ ]:
example_covariates = ["black", "urban", "test"]
df[example_covariates].describe()

In [ ]:
sns.countplot(x="treat", data=df)

<img src="material/fig-catholic-school-example.png" height=500 width=500 />

Is there any hope in identifying the $ATE$?

<img src="material/fig-matching-demonstration-four-propensity-score.png" height=500 width=500 />

<img src="material/fig-matching-demonstration-four-potential-outcome.png" height=500 width=500 />

Here comes the key feature that generates the dependence between $D$ and $Y$ based on an unobservable.

\begin{align*}
y_i^1 = y_i^0 + \delta^\prime_i + \delta^{\prime\prime}_i
\end{align*}

$\rightarrow$ $\delta^{\prime\prime}_i$ is a associated with one of the potential outcomes and also affects the probability to select treatment.

However, we can still identify the $ATT$. Why?

\begin{align*}
E[\delta \mid D = 1, S] & = E[Y^1 - Y^0 \mid D = 1, S] \\
                        & = E[Y^1 \mid D = 1, S] - E[Y^0 \mid D = 1, S] \\
                        & = E[Y^1 \mid D = 1, S] - E[Y^0 \mid D = 0, S] \\
                        & = E[Y \mid D = 1, S] - E[Y \mid D = 0, S] \\
\end{align*}

In [ ]:
stat = (df["yt"] - df["yc"])[df["treat"] == 1].mean()
print("The true ATT is {:5.3f}".format(stat))

In [ ]:
def nearest_neighbor_algorithm_for_att(df, check_store=True):

    if check_store:
        if os.path.exists("matched.ngbr.pkl"):
            return pkl.load(open("matched.ngbr.pkl", "rb"))

    # We select all treated individuals
    df_control = df[df["treat"] == 0]
    df_treated = df[df["treat"] == 1]

    rslt = np.full((df_treated.shape[0], 11), np.nan)

    # We now iterate over all treated individuals and
    # find a set of neighbors.
    for i, (index, row) in enumerate(df_treated.iterrows()):

        y, p, b, u, t = row[["y", "p", "black", "urban", "test"]]
        df_control = df_control.assign(distance=np.abs(df_control["p"] - p))

        idx_ngbr = df_control["distance"].idxmin()
        y_ngbr, p_ngbr = df_control.loc[idx_ngbr, ["y", "p"]]
        b_ngbr, u_ngbr = df_control.loc[idx_ngbr, ["black", "urban"]]
        t_ngbr = df_control.loc[idx_ngbr, ["test"]]

        rslt[i] = [i, y, y_ngbr, p, p_ngbr, b, b_ngbr, u, u_ngbr, t, t_ngbr]

    columns = ["count", "y", "y_ngbr", "p", "p_ngbr", "b", "b_ngbr"]
    columns += ["u", "u_ngbr", "t", "t_ngbr"]
    df = pd.DataFrame(rslt, columns=columns)

    pkl.dump(df, open("matched.ngbr.pkl", "wb"))

    return df


df = get_sample_matching_demonstration_4()
df["p"] = get_propensity_scores_matching_demonstration_4(df)
get_common_support(df, "treat")

In [ ]:
df_matched = nearest_neighbor_algorithm_for_att(df, False)
sns.jointplot("p", "p_ngbr", df_matched)

In [ ]:
sns.jointplot("y", "y_ngbr", df_matched)

How do our covariantes balance across treatment status?

In [ ]:
df.groupby("treat")[example_covariates].mean().T

We now want to revisit the balancing of covariates.

In [ ]:
rename = {
    "y_ngbr": "y",
    "p_ngbr": "p",
    "b_ngbr": "b",
    "u_ngbr": "u",
    "t_ngbr": "t",
}

df_control = df_matched[["y_ngbr", "p_ngbr", "b_ngbr", "u_ngbr", "t_ngbr"]]
df_control = df_control.rename(columns=rename)
df_control = df_control.assign(treat=0)

df_treated = df_matched[["y", "p", "b", "u", "t"]]
df_treated = df_treated.assign(treat=1)

df_subset = pd.concat([df_treated, df_control])
df_subset.groupby("treat").mean().T

Let's take a little detour and look at the balancing of observables in the Lalonde dataset.

In [ ]:
df = get_lalonde_data()
df.head()

In [ ]:
example_covariates = ["black", "married", "hispanic", "re75"]
df.groupby("treat")[example_covariates].mean().T

The covariates are balanced before any reweighting thanks to the assignment mechanisms.



Which matching algorihtm is the best?

<img src="material/fig-matching-demonstration-four-benchmarking.png" height=500 width=500 />